In [ ]:
#create and train the cnn model

import cv2
from keras.models import load_model
import os
import pandas as pd
import numpy as np
import pygame
from phue import Bridge
from soco import SoCo
from keras.models import Sequential
from keras import layers
import keras.backend as K
from keras.layers import Dropout, Activation
from keras.layers import Input, ZeroPadding2D, Flatten, Conv2D
from keras.layers import Dense, BatchNormalization, MaxPooling2D, GlobalMaxPooling2D
from keras.layers import AveragePooling2D, GlobalAveragePooling2D
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint, EarlyStopping


def createCNNModel(imX, imY):
    model = Sequential()
    model.add(Conv2D(32, (5, 5), activation='relu', input_shape=(imX, imY, 1)))
    model.add(MaxPooling2D(2,2))
    model.add(Conv2D(64, (5, 5), activation='relu'))#sigmoid
    model.add(MaxPooling2D(5,5))
    model.add(Flatten())
    model.add(Dense(1024, activation='relu'))
    model.add(Dropout(0.25))
    numEmotes = 8
    model.add(Dense(numEmotes, activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])
    callbacks_list = [ModelCheckpoint(filepath='R-Emote_Model.h5', monitor='val_acc', verbose=1, save_best_only=True, mode='max')]
    return model, callbacks_list


'''def trainCNN():
    data = pd.read_csv("trainingData.csv")
    dataset = np.array(data)
    np.random.shuffle(dataset)
    X = dataset
    Y = dataset
    X = X[:, 1:2501]
    Y = Y[:, 0]

    X_train = X[0:10501, :]
    X_train = X_train / 255.
    X_test = X[9299:10501, :]
    X_test = X_test / 255.

    # Reshape
    Y = Y.reshape(Y.shape[0], 1)
    Y_train = Y[0:10501, :]
    Y_train = Y_train.T
    Y_test = Y[9299:10501, :]
    Y_test = Y_test.T
    
    print("number of training examples = " + str(X_train.shape[0]))
    print("number of test examples = " + str(X_test.shape[0]))
    print("X_train shape: " + str(X_train.shape))
    print("Y_train shape: " + str(Y_train.shape))
    print("X_test shape: " + str(X_test.shape))
    print("Y_test shape: " + str(Y_test.shape))
    image_x = 50
    image_y = 50
    
    train_y = np_utils.to_categorical(Y_train)
    test_y = np_utils.to_categorical(Y_test)
    
    train_y = train_y.reshape(train_y.shape[1], train_y.shape[2])
    test_y = test_y.reshape(test_y.shape[1], test_y.shape[2])
    X_train = X_train.reshape(X_train.shape[0], 50, 50, 1)
    X_test = X_test.reshape(X_test.shape[0], 50, 50, 1)
    print("X_train shape: " + str(X_train.shape))
    print("X_test shape: " + str(X_test.shape))

    model, callbacks_list = createCNNModel(image_x, image_y)
    model.fit(X_train, train_y, validation_data=(X_test, test_y), epochs=10, batch_size=64, callbacks=callbacks_list)
    scores = model.evaluate(X_test, test_y, verbose=0)
    print("Error in CNN: %.2f%%" % (100 - scores[1] * 100))
    print('Accuracy of CNN: %.2f%%' % ((scores[1])*100))
    #return model
    model.save('R-Emote_Model.h5')'''
    
def gatherEmotes():
    emotesDir = 'EmoteMePls/emoticons/'
    emotes = []
    print(len(os.listdir(emotesDir)))
    for emote in range((len(os.listdir(emotesDir)))):
        print(emote)
        emotes.append(cv2.imread(emotesDir+str(emote)+'.png', -1))
    return emotes

def predict(CNNModel, newImage):
    processed = process(newImage)
    pred_probab = CNNModel.predict(processed)[0]
    pred_class = list(pred_probab).index(max(pred_probab))
    processTask(pred_class)
    predValue = float(CNNModel.predict(processed, batch_size=1))
    predValue = steering_angle * 100
    return max(pred_probab), pred_class

def pygameInit():
    pygame.init()
    pygame.mixer.init()
    pygame.mixer.music.load('/Users/shahzaibparacha/Downloads/No More Parties in LA.mp3')
    pygame.mixer.music.set_volume(0.5)
    pygame.mixer.music.pause()
        
def PhilipsHueInit():
    bridge_ip = '192.168.1.127'
    b = Bridge(bridge_ip)
    on_command = {'transitiontime': 0, 'on': True, 'bri': 254}
    off_command = {'transitiontime': 0, 'on': False, 'bri': 254}

def SonosInit():
    sonos_ip = '192.168.0.3'
    myRoom = SoCo(sonos_ip)
    myRoom.status_light = True
    myRoom.volume = 7
    myRoom.play_uri('https://soundcloud.com/kanyewest/nomorepartiesinla')

def processTask(predcition):
    event = ''
    if automateTasks:
            if prediction == 1:
                try:
                    event = "Play music"
                    myRoom.play()
                    pygame.mixer.music.unpause()
                except ConnectionError:
                    smart_home = False
                    pass

            elif prediction == 2:
                try:
                    event = 'Lights off'
                    b.set_light(6, off_command)
                except ConnectionError:
                    smart_home = False
                    pass

            elif prediction == 3:
                try:
                    event = "Lights on"
                except ConnectionError:
                    smart_home = False
                    pass

            elif prediction == 4:
                try:
                    event = 'Volume up'
                    pygame.mixer.music.set_volume(pygame.mixer.music.get_volume() + 0.15)
                except ConnectionError:
                    smart_home = False
                    pass

            elif prediction == 5:
                try:
                    event = 'Volume down'
                    pygame.mixer.music.set_volume(pygame.mixer.music.get_volume() - 0.15)
                except ConnectionError:
                    smart_home = False
                    pass
                
            elif prediction == 6:
                try:
                    event = ''
                except ConnectionError:
                    smart_home = False
                    pass
                
            elif prediction == 7:
                try:
                    event = 'Pause music'
                    myRoom.pause()
                    pygame.mixer.music.pause()
                except ConnectionError:
                    smart_home = False
                    pass

            else:
                pass

def process(frame):
    frame = cv2.resize(frame, (50, 50))
    frame = np.array(frame, dtype=np.float32)
    frame = np.reshape(frame, (-1, 50, 50, 1))
    return frame
    
    
                          


def overlayImg(image, emoji, x,y,w,h):
    emoji = cv2.resize(emoji, (w, h))
    try:
        image[y:y+h, x:x+w] = blend_transparent(image[y:y+h, x:x+w], emoji)
    except:
        pass
    return image

def blend_transparent(face_img, overlay_t_img):
    overlay_img = overlay_t_img[:,:,:3] # Grab the BRG planes
    overlay_mask = overlay_t_img[:,:,3:]  # And the alpha plane

    background_mask = 255 - overlay_mask

    overlay_mask = cv2.cvtColor(overlay_mask, cv2.COLOR_GRAY2BGR)
    background_mask = cv2.cvtColor(background_mask, cv2.COLOR_GRAY2BGR)

    face_part = (face_img * (1 / 255.0)) * (background_mask * (1 / 255.0))
    overlay_part = (overlay_img * (1 / 255.0)) * (overlay_mask * (1 / 255.0))

    return np.uint8(cv2.addWeighted(face_part, 255.0, overlay_part, 255.0, 0.0))



#trainCNN()
print('works')
#model = trainCNN()
model = load_model('R-Emote_Model.h5.h5')
#processAndPredict(model, np.zeros((50, 50, 1), dtype=np.uint8))
print('works')
if __name__ == "__main__":
    automateTasks = true
    PhilipsHueInit()
    SonosInit()
    pygameInit()
    outputPred = ''
    emotes = gatherEmotes()
    #model = trainCNN()
    captureImage = cv2.VideoCapture(0)
    top, right, bottom, left = 300, 50, 350, 350
    x, y, w, h = 300, 50, 350, 350
    
    while (captureImage.isOpened()):
        grabbed, frame = captureImage.read()
        #frame = cv2.flip(frame, 1)
        #hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        #altMasking = cv2.inRange(hsv, np.array([2, 50, 60]), np.array([25, 150, 255]))
        #roi = cv2.bitwise_and(frame, frame, mask=altMasking)
        frame = cv2.flip(frame, 1)
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        setGray = cv2.GaussianBlur(gray, (5, 5), 0)
        #gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
        #setGray = cv2.GaussianBlur(gray, (5, 5), 0)
        grabbed, thresholded = cv2.threshold(setGray, 150, 255, cv2.THRESH_BINARY)
        
        
        setSquare = np.ones((5, 5), np.uint8)
        setDilation = cv2.dilate(setGray, setSquare, iterations=2)
        setOpening=cv2.morphologyEx(setDilation,cv2.MORPH_CLOSE,setSquare)
        #grabbed, thresholded = cv2.threshold(setOpening, 30, 255, cv2.THRESH_BINARY_INV)
        
        #thresholded = thresholded[right:right + left, top:top + bottom]
        thresholded = thresholded[y:y + h, x:x + w]
        (cnts,_) = cv2.findContours(thresholded.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)[-2:]
        if len(cnts) > 0:
            cnt = max(cnts, key=cv2.contourArea)
            if cv2.contourArea(cnt) > 2500:
                Top, Right, ptBottom, ptLeft = cv2.boundingRect(cnt)
                newImg = thresholded[Right:Right + ptLeft, Top:Top + ptBottom]
                newImg = cv2.resize(newImg, (50, 50))
                confidencePred, classPred = predict(model, newImg)
                print(confidencePred, classPred)
                frame = overlayImg(frame, emotes[classPred], 400, 250, 90, 90)
                outputPred = "Accuracy = "+str(predValue)+'%'
        #top, right, bottom, left = 300, 50, 350, 350
        cv2.rectangle(frame, (top, right), (top + bottom, right + left), (0, 255, 0), 2)
        cv2.putText(frame, outputPred, (30, 60), cv2.FONT_HERSHEY_DUPLEX, 2, (127, 255, 255))
        cv2.imshow("Frame", frame)
        cv2.imshow("Contours", thresholded)
        k = cv2.waitKey(10)
        if k == 32:
            break
print('works')